In [231]:
import numpy as np 
import pandas as pd
from scipy.optimize import minimize

In [ ]:
test_maturity = [i * 0.5 for i in range(1,12)]
test_price = [99.1338, 97.8925, 96.1462, 94.1011, 91.7136, 89.2258, 86.8142, 84.5016, 82.1848, 79.7718, 77.4339]
test_yields = [1.74, 2.13, 2.62, 3.04, 3.46, 3.80, 4.04, 4.21, 4.36, 4.52, 4.65]
test_yields = np.array(yields) / 100

array([0.0174, 0.0213, 0.0262, 0.0304, 0.0346, 0.038 , 0.0404, 0.0421,
       0.0436, 0.0452, 0.0465])

In [232]:
def Building_Tree(theta,
                  number_of_periods = yield_curve['Yield'].to_numpy().shape[0],
                  r0 = r0,
                  volatility = rate_vol,
                  dt = dt):

 
 #Creating Binomial Tree of length i, height i 
 binomial_tree = np.zeros((number_of_periods , number_of_periods)) 
 binomial_tree[np.triu_indices(number_of_periods, 0)] = r0

 #Setting labels and tags to make binomial tree more readable
 binomial_tree = pd.DataFrame(binomial_tree)
 binomial_tree.columns = [i for i in range(0, number_of_periods)]
 binomial_tree.index = [i for i in range(0,number_of_periods)]
 binomial_tree.columns.name = 'i'
 binomial_tree.index.name = 'j'
 
 #Apply Ho-Lee
 
 for i in range(1, number_of_periods): #Time steps from i = 1,2, ... N-1
      for j in range(i + 1):      #States from 0 to i 
         
            if j == 0:
                previous_rate = binomial_tree.iloc[j, i-1]
                binomial_tree.iloc[j , i] = previous_rate + theta[i-1]*dt + volatility*np.sqrt(dt)
            else:
                previous_rate = binomial_tree.iloc[j-1, i-1]
                binomial_tree.iloc[j, i] = previous_rate + theta[i-1]*dt - volatility*np.sqrt(dt)

 return binomial_tree

In [233]:
def ZCB(yield_curve=yield_curve['Yield'].to_numpy(), dt = dt, periods = 50):
    
    if isinstance(yield_curve, np.ndarray):
        yield_curve = list(yield_curve)
        
    ZCB_prices = np.zeros(periods)
    
    for i in range(periods):
        discount_factor = np.exp( - yield_curve[i] * (dt*(i+1)))
        ZCB_prices[i] = 100.00 * discount_factor
    
    return ZCB_prices


In [234]:
def Ho_Lee_Model_LossFunction(theta, periods, r0 = r0, volatility = rate_vol, dt = dt, 
                              yield_curve = yield_curve['Yield']):
    '''  
    This calculates the error function between the Ho-lee model (using initial theta values that 
    are basically random values) and the actual observed ZCB prices calculated from ZCB() function.

    The idea is to use this function in tandem with an optimiser that will fit the theta values by
    minimising this loss function.

    Not the most computationally efficient method but shows the information flow well.

    Loss function used is MSE.
    Uses the short-rate at time = 0 as an initial value (r0)
    '''
    Error_SSE = 0.
    ZCBx = ZCB(yield_curve=yield_curve, periods = periods)

    for period in range(1, periods + 1):
        target_model_price = ZCB_Ho_Lee_Calibration(maturity = period - 1, theta = theta, r0 = r0)
        observed_price = ZCBx[period - 1]
        Error_SSE += (target_model_price - observed_price)**2

    return Error_SSE


In [ ]:
def ZCB_Ho_Lee_Calibration(maturity, theta, r0 = r0, dt = dt, par_value = 100, volatility = rate_vol, test=False):
    ''' 
    To calibrate Ho-Lee model, we use backward induction to calculate the price of ZCB bonds.
    This function will be passed through to an optimiser, we can minimise some loss function by comparing this to the true observed
    ZCB prices. This function manually sets the final layer based on the payoff

    Risk neutral probabilities uses q = 0.5
    Parameters :
        1. maturity : ZCB for a specific maturity 
        2. theta : theta vector
        3. dt : set to 0.5 years
        4. par value : $100 ZCB 
    ----------
    Returns ZCB_price : present value of cash flows e.g., price of ZCB bond at t = 0
    
    '''
    periods = maturity + 1
    #Final node is based on the defined payoff below (for ZCB it's simple)
    binomial_tree = Building_Tree(theta, number_of_periods=periods, dt=dt, volatility=volatility, r0 = r0)
    
    #Create Value Tree and set last row to $100
    value_tree = pd.DataFrame(np.zeros(periods * periods).reshape(periods,periods)).reindex_like(binomial_tree)

    #Penultimate Node
    value_tree.iloc[:,-1] = par_value * np.exp(-binomial_tree.iloc[:,-1] * dt)

    #Backward Induction : start from final node, go backwards to i = 0 period
    for i in range(periods-2, -1, -1):

        #Only goes up to when i = j (diagonal)
        for j in range(i+1): 
            #Discounted Expected Value of the bond at time i
            value_tree.iloc[j, i] = 0.5 * (value_tree.iloc[j, i+1] + value_tree.iloc[j+1, i+1]) * np.exp(-binomial_tree.iloc[j, i] * dt)
    
    if test == True:
        return value_tree.iloc[0,0], value_tree
    else:
        return value_tree.iloc[0,0]

In [ ]:
def calibration_HL(yield_curve = yield_curve['Yield'].to_numpy(), periods = 50, sigma = rate_vol, dt = dt, r0 = yield_curve['Yield'].iloc[0]):
    ''' 
    argmin theta vector of norm-2 errors 

    '''
    initial_guess_theta = np.ones(periods) * 0.025

    #objective function 
    def objective_function(theta):
        return Ho_Lee_Model_LossFunction(theta=theta, periods = periods, r0 = r0, yield_curve=yield_curve)
    
    result = minimize(objective_function, initial_guess_theta, method = 'BFGS',
                      options={
                          'maxiter': 5000,
                          'gtol' : 0.005,
                          'eps': 0.00001,
                          'disp': True
                      })
    print(result.success, result.message)

    theta_calibrated = result.x

    return theta_calibrated, result
